In [3]:
# 导入库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 处理中文字体
from matplotlib import rcParams
rcParams['font.family']='Microsoft YaHei'

# 导入数据
df=pd.read_csv('data/house_sales.csv')
df.head()

,city,address,area,floor,name,price,province,rooms,toward,unit,year,origin_url
0,合肥,龙岗-临泉东路和王岗大道交叉口东南角,90㎡,中层（共18层）,圣地亚哥,128万,安徽,3室2厅,南北向,14222元/㎡,2013年建,https://hf.esf.fang.com/chushou/3_404230646.htm
1,合肥,龙岗-临泉东路和王岗大道交叉口东南角,90㎡,中层（共18层）,圣地亚哥,128万,安徽,3室2厅,南北向,14222元/㎡,2013年建,https://hf.esf.fang.com/chushou/3_404230646.htm
2,合肥,生态公园-淮海大道与大众路交口,95㎡,中层（共18层）,正荣·悦都荟,132万,安徽,3室2厅,南向,13895元/㎡,2019年建,https://hf.esf.fang.com/chushou/3_404304901.htm
3,合肥,生态公园-淮海大道与大众路交口,95㎡,中层（共18层）,正荣·悦都荟,132万,安徽,3室2厅,南向,13895元/㎡,2019年建,https://hf.esf.fang.com/chushou/3_404304901.htm
4,合肥,撮镇-文一名门金隅裕溪路与东风大道交口,37㎡,中层（共22层）,文一名门金隅,32万,安徽,2室1厅,南北向,8649元/㎡,2017年建,https://hf.esf.fang.com/chushou/3_404372096.htm


In [ ]:
# 数据清洗
# 删除无用的数据 url
df.drop(columns='origin_url',inplace=True)

In [11]:
# 删除缺失值
df.dropna(inplace=True)

In [13]:
# 重复数据
df.duplicated().sum()

np.int64(29416)

In [14]:
# 删除重复数据
df.drop_duplicates(inplace=True)

In [ ]:
# 数据类型的转换
# 面积数据类型的转换，字符串转化为float
df['area']=df['area'].str.replace('㎡','').astype(float)


# 价格数据类型的转换，字符串转化为float
df['price']=df['price'].str.replace('万','').astype(float)


# 朝向的数据类型转换
df['toward']=df['toward'].astype('category')


# 单价的数据类型转换
df['unit']=df['unit'].str.replace('元/㎡','').astype(float)


# 建造年份的数据类型转换
df['year']=df['year'].str.replace('年建','').astype(int)

In [41]:
# 异常值的处理
# 房屋面积的异常处理
df=df[(df['area']<600) & (df['area']>20)]


# 房屋售价的异常值处理 IQR
Q1=df['price'].quantile(0.25)
Q3=df['price'].quantile(0.75)
IQR=Q3-Q1
low_price=Q1-1.5*IQR
high_price=Q3+1.5*IQR
df=df[(df['price']>low_price) & (df['price']<high_price)]

In [ ]:
# 新数据特征构造

# 地区district
# 楼层类型floor_type
# 是否是直辖市zxs
# 卧室数量bedrooms
# 客厅数量livingrooms
# 楼龄building_age
# 价格的分段Price_labels

In [44]:
df.head()

,city,address,area,floor,name,price,province,rooms,toward,unit,year
0,合肥,龙岗-临泉东路和王岗大道交叉口东南角,90.0,中层（共18层）,圣地亚哥,128.0,安徽,3室2厅,南北向,14222.0,2013
2,合肥,生态公园-淮海大道与大众路交口,95.0,中层（共18层）,正荣·悦都荟,132.0,安徽,3室2厅,南向,13895.0,2019
4,合肥,撮镇-文一名门金隅裕溪路与东风大道交口,37.0,中层（共22层）,文一名门金隅,32.0,安徽,2室1厅,南北向,8649.0,2017
6,合肥,龙岗-长江东路与和县里交口,50.0,高层（共30层）,柏庄金座,46.0,安徽,2室1厅,南向,9200.0,2019
8,合肥,新亚汽车站-张洼路与临泉路交汇处向北100米(原红星机械,120.0,中层（共27层）,天目未来,158.0,安徽,4室2厅,南向,13167.0,2012
